In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
%matplotlib inline
sns.set_context("poster",rc={"font.size": 28})
sns.set_style("whitegrid")

In [13]:
imaclimfolder = "C:/Users/WB451855/OneDrive - WBG/!infra_needs_data/transport_cired_output_data/outputs_Imaclim/"
itfdatafolder = "C:\\Users\\WB451855\\OneDrive - WBG\\!infra_needs_data\\transport_itf_output_data\\"
coastalprotection = "C:/Users/WB451855/OneDrive - WBG/!infra_needs_data/coastal_protection_output_data/"
irrigationfolder = "C:\\Users\\WB451855\OneDrive - WBG\\!infra_needs_data\\water_iiasa_output_data\\"
energy_iiasa = "C:\\Users\\WB451855\OneDrive - WBG\\!infra_needs_data\\\energy_iiasa_output_data\\"
riverfloodprotection = "C:/Users/WB451855/OneDrive - WBG/!infra_needs_data/river_floods/"

In [3]:
all_ima_data = pd.DataFrame()
for myfilename in os.listdir(imaclimfolder):
    if "WORLD" in myfilename:
        continue
    myfile = pd.read_csv(imaclimfolder+"\\"+myfilename,sep=";")
    myinfo = myfilename.replace(".csv",'').split("_")
    myfile["region"] = myinfo[3]
    all_ima_data = all_ima_data.append(myfile)

In [4]:
indexes_ima = ['//combi', 'ind_climat', 'ind_growth', 'ind_mitig', 'ind_trans_A',
       'ind_trans_S', 'ind_trans_I', 'ind_trans_F', 'ind_taxexo', 'ind_recycl',
       'ind_taxmax', 'Variable', 'Unit','region']

all_ima_data = all_ima_data.set_index(indexes_ima)
all_ima_data.columns.name='year'
all_ima_data = all_ima_data.drop(['2001', '2002', '2003', '2004',
       '2005', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014','2050', '2051', '2052', '2053', '2054', '2055', '2056', '2057', '2058',
       '2059', '2060', '2061', '2062', '2063', '2064', '2065', '2066', '2067',
       '2068', '2069', '2070', '2071', '2072', '2073', '2074', '2075', '2076',
       '2077', '2078', '2079', '2080', '2081', '2082', '2083', '2084', '2085',
       '2086', '2087', '2088', '2089', '2090', '2091', '2092', '2093', '2094',
       '2095', '2096', '2097', '2098', '2099'],axis=1)
all_ima_data = all_ima_data.stack('year')
all_ima_data.name = 'value'

In [5]:
indexes_ima2 = ['ind_climat', 'ind_growth', 'ind_mitig', 'ind_trans_A',
       'ind_trans_S', 'ind_trans_I', 'ind_trans_F', 'ind_taxexo', 'ind_recycl',
       'ind_taxmax', 'Variable','region','year']

In [6]:
imaclim = all_ima_data.reset_index().drop(['Unit','//combi'],axis=1).set_index(indexes_ima2).unstack('Variable').value

In [7]:
itf = pd.read_csv(itfdatafolder+"country_data.csv",encoding='latin-1')

In [14]:
energy = pd.read_excel(energy_iiasa+"analysis IIASA august.xlsx","discounted",usecols=range(0,9))

In [15]:
energy_gdp = energy.loc[energy.Variable=="GDP|MER",:].set_index(["Model","Region","Scenario","Unit"]).drop("Variable",axis=1)
energy_gdp.columns.name="Year"

In [16]:
energy_gdp = energy_gdp.stack().reset_index().rename(columns={0:"GDP"})

In [17]:
energy_gdp["discount_factor"] = (1+0.06)**(1-(energy_gdp.Year.astype(int)-2015))
energy_gdp['disc_PIB'] =  energy_gdp['GDP']*energy_gdp["discount_factor"]

In [18]:
energy_gdp.reset_index().Region.unique()

array(['R5ASIA', 'R5LAM', 'R5MAF', 'R5REF', 'World', 'Developing'], dtype=object)

In [19]:
energy_gdp.loc[(energy_gdp.Year<2031)&(energy_gdp.Scenario=="2C"),:].set_index(["Model","Region","Year"]).disc_PIB.sum(level=["Model","Region"])\
.drop(["CHN","IND","R5OECD90+EU","USA","World","EU"],level="Region").sum(level="Model")

Model
AIM/CGE              169676.176066
IMAGE                221709.147612
MESSAGEix-GLOBIOM    253913.595388
POLES                176924.945476
REMIND-MAgPIE        191825.354127
WITCH-GLOBIOM        179395.735872
Name: disc_PIB, dtype: float64

In [20]:
pib_itf = itf.loc[itf.GDP_USD2010!="null",["ISO3","Year","GDP_USD2010"]].set_index(["ISO3","Year"])

In [21]:
pib_cired = imaclim.PIB

In [22]:
#USD2005. USD2015 = 1.21*USD2005
1.21*pib_cired.unstack("year")["2015"].mean(level="region")

region
ASIA    11920.004097
CIS      2266.556875
LAM      5671.211181
MAF      4353.340521
OCDE    51278.094236
Name: 2015, dtype: float64

In [31]:
energy_gdp.loc[(energy_gdp.Year==2015)&(energy_gdp.Scenario=="2C"),:].set_index(["Model","Region","Year"]).drop(["CHN","IND","R5OECD90+EU","USA","World","EU","Developing"],level="Region").GDP

Model              Region  Year
AIM/CGE            R5ASIA  2015    12405.650744
                   R5LAM   2015     3929.737618
                   R5MAF   2015     3543.862890
                   R5REF   2015     1486.310836
IMAGE              R5ASIA  2015    15449.843880
                   R5LAM   2015     4787.256179
                   R5MAF   2015     3717.199168
                   R5REF   2015     1747.893993
MESSAGEix-GLOBIOM  R5ASIA  2015    16863.249603
                   R5LAM   2015     4717.022169
                   R5MAF   2015     4369.313765
                   R5REF   2015     2614.204282
POLES              R5ASIA  2015    13083.301844
                   R5LAM   2015     4656.388481
                   R5MAF   2015     3948.574472
                   R5REF   2015     1265.545986
REMIND-MAgPIE      R5ASIA  2015    14411.760686
                   R5LAM   2015     4960.937125
                   R5MAF   2015     4083.313543
                   R5REF   2015     1354.913434
WITCH-GL

In [15]:
pib_itf.unstack("Year")["GDP_USD2010"][2015].astype(float)/1000000000

ISO3
ABW        0.206229
AFG       45.210103
AGO      130.758335
ALB       27.122767
ARE      374.757131
ARG      689.640513
ARM       19.666361
ATG        1.662126
AUS      925.570630
AUT      320.978166
AZE       97.505469
BDI        5.453943
BEL      375.573032
BEN       15.752583
BFA       25.951852
BGD      299.777766
BGR       93.107176
BHR       31.729086
BHS       10.802656
BIH       29.923194
BLR      137.290051
BLZ        2.382074
BOL       55.466481
BRA     2259.830996
BRB        6.431774
BRN       21.936224
BTN        5.315485
BWA       31.839910
CAF        3.013426
CAN     1367.786972
           ...     
SWZ        5.600613
SYC        2.276468
SYR      118.584213
TCD       20.081514
TGO        7.134902
THA      644.101283
TJK       18.526595
TKM       63.723338
TLS        2.241386
TON        0.474890
TTO       32.689404
TUN      101.840349
TUR     1166.872849
TUV        0.035000
TWN      863.475549
TZA       78.572991
UGA       50.965754
UKR      301.604087
URY       51.73

In [16]:
1.09*pib_itf.unstack("Year")["GDP_USD2010"][2015].astype(float).sum()/1000000000

88285.65190047208

In [17]:
1.21*pib_cired.unstack("year")["2015"].mean(level="region").sum()

75489.20690972223

In [38]:
#USD2005 PPP
rivers = pd.read_csv(riverfloodprotection+"region_gdp.csv")

In [42]:
10**(-9)*rivers.loc[rivers.year==2015,:].set_index(["wbregionname","ssp"]).gdp.mean(level="wbregionname")

wbregionname
East Asia & Pacific           19523.584703
Europe & Central Asia          3352.766303
High Income                   43804.024907
Latin America & Caribbean      6992.057854
Middle East & North Africa     2901.401600
South Asia                     7036.326707
Sub-Saharan Africa             2594.773023
Name: gdp, dtype: float64

In [32]:
#USD2014
coastal = pd.read_csv(coastalprotection+"region_output_WB.csv")

In [33]:
coastal = coastal.loc[(coastal.ssp=="SSP2")&(coastal.time==2015)&(coastal.rcp=="RCP 4.5 (Medium ice melting)")&(coastal.ada=="CBA Protection"),:]

In [34]:
coastal = coastal.set_index(['locationid', 'locationname','rcp', 'ada', 'ssp', 'adacost','riverprotection',"time"])

In [38]:
coastal.gdp.mean(level="locationid")

locationid
EAP    2.205779e+07
ECA    3.525881e+06
LAC    8.422365e+06
MNA    2.848716e+06
SAS    7.632810e+06
SSA    2.896117e+06
YHI    5.676982e+07
Name: gdp, dtype: float64

In [20]:
1/1000000*coastal.gdp.unstack("time")[2015].mean(level=["locationid","ssp"]).sum(level="ssp")

ssp
SSP2    104.153499
SSP3    103.969394
SSP5    104.285884
Name: 2015, dtype: float64

In [21]:
#2015 global GDP in USD2015
76*1.09

82.84

In [22]:
83/104

0.7980769230769231